In [1]:
import torch
import gluonnlp as nlp                  # GluonNLP는 버트를 간단하게 로딩하는 인터페이스를 제공하는 API 임
import numpy as np

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import BertTokenizer, BertModel

from utils import seed_everything, GPU_info, pytorch_cos_sim


In [2]:
#kobert 다운받아서 압축푼 폴더(config.json, pytorch_model.bin 같이 있는 폴더 지정)
output_hidden_states = True # 기본은 False=>output 2개 출력됨, True로 지정하면 output이 3개 출력됨
return_dict = False
model_path = "model/kobertmodel" 
model = BertModel.from_pretrained(model_path, output_hidden_states = output_hidden_states, return_dict = return_dict)
model.eval()
model.num_parameters()

92186880

In [5]:
vocab_file="Tokenizer/kobert/kobert_news_wiki_ko_cased-ae5711deb3.spiece"
vocab = nlp.vocab.BERTVocab.from_sentencepiece(vocab_file, padding_token="[PAD]")
tokenizer = nlp.data.BERTSPTokenizer(vocab_file, vocab, lower=False)

# pad는 원래 True로 했는데, 여기서는 False로 해도 마찬가지 유사도 결과가 나옴
transform = nlp.data.BERTSentenceTransform(
            tokenizer, max_seq_length = 128, pad=False, pair=False)

In [14]:
test_sentence = ['식당에 가서 밥을 배 부르게 먹고 낙시배를 타고 고기 잡고 요트배를 타고 관광을 해야 겠다']

transform_data = [transform([i[0]]) for i in [test_sentence]]
#print(transform_data)
#print(type(transform_data))

token_ids = transform_data[0][0]
valid_length = transform_data[0][1]
segment_ids = transform_data[0][2]


#print("token_ids:{}".format(token_ids))
#print("segment_ids:{}".format(segment_ids))
#print("valid_length:{}".format(valid_length))

#print(type(token_ids))

#tensor로 변환[] 묶어줘서 무조건 2차원으로 출력해야 함
token_ids = torch.tensor([token_ids])
segment_ids = torch.tensor([segment_ids])

print("token_ids:\r\n{}".format(token_ids))
print("segment_ids:\r\n{}".format(segment_ids))

# attention mask 설정
# => 토큰에 대해서는 1 PAD 토큰은 0으로 설정
attention_mask = torch.zeros_like(token_ids)
for v in range(valid_length):
  attention_mask[0][:v] = 1
attention_mask.float()

print('attention_mask:\r\n{}'.format(attention_mask))

token_ids_list = token_ids.tolist()
#print(token_ids_list[0])

count = 0
for token in token_ids_list[0]:
    print('{}: {}    {}'.format(count, vocab.idx_to_token[token],token))
    count+=1

token_ids:
tensor([[   2, 3007, 6896,  517, 5330, 6553, 2266, 7088, 2287, 2432, 5400, 2011,
         1404, 6705, 6312, 6116, 4700,  993, 5561, 3951, 3480, 7659, 6312, 6116,
         4700, 1080, 7088, 5010,  517, 5406,    3]], dtype=torch.int32)
segment_ids:
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]], dtype=torch.int32)
attention_mask:
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 0]], dtype=torch.int32)
0: [CLS]    2
1: ▁식당    3007
2: 에    6896
3: ▁    517
4: 가    5330
5: 서    6553
6: ▁밥    2266
7: 을    7088
8: ▁배    2287
9: ▁부르    2432
10: 게    5400
11: ▁먹고    2011
12: ▁낙    1404
13: 시    6705
14: 배    6312
15: 를    6116
16: ▁타고    4700
17: ▁고    993
18: 기    5561
19: ▁잡고    3951
20: ▁요    3480
21: 트    7659
22: 배    6312
23: 를    6116
24: ▁타고    4700
25: ▁관광    1080
26: 을    7088
27: ▁해야    5010
28: ▁    517
29: 겠다    5406
30: [SEP]    3


In [8]:
#device = torch.device("cpu")

#bertmodel.eval()  #모델을 평가모드로 변경후

output = model(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float())
output_len = len(output)
print('출력 out 길이:{}'.format(output_len))

sequence_output = output[0]  # tensor 임
pooled_output = output[1]    # tensor 임

print('sequence 길이: {}, pooled 길이:{}'.format(sequence_output.size(), pooled_output.size()))

# output_len 길이가 2이상이면 hidden_states 출력함
if output_len > 2:
  hidden_states = output[2]  #hidden state 출력 (tuple임)
  # tuple 은 size로 출력 못하므로, 아래 처럼 출력함
  layer_idx = 0
  batch_idx = 0
  token_idx = 0
  print('hidden_states')
  print("-레이어 수:{}".format(len(hidden_states)))
  print("-배치 수: {}".format(len(hidden_states[layer_idx])))
  print("-토큰 수 : {}".format(len(hidden_states[layer_idx][batch_idx])))
  print("-hidden 유닛 수 : {}".format(len(hidden_states[layer_idx][batch_idx][token_idx])))

출력 out 길이:3
sequence 길이: torch.Size([1, 31, 768]), pooled 길이:torch.Size([1, 768])
hidden_states
-레이어 수:13
-배치 수: 1
-토큰 수 : 31
-hidden 유닛 수 : 768


In [13]:
# premute를 사용하여 레이어 와 tokens 차원을 바꾼다.
sequence_output_embedding = sequence_output.permute(1,0,2)
print('sequence_output size: {}'.format(sequence_output_embedding.size()))

# tensor -> list 로 변환
output_list = sequence_output_embedding.tolist()

# 몸에 있는배, 낙시배 비교
simulate1 = pytorch_cos_sim(output_list[8][0], output_list[14][0])

# 몸에 있는배, 요트배 비교
simulate2 = pytorch_cos_sim(output_list[8][0], output_list[22][0])

# 낙시배, 요트배 비교
simulate3 = pytorch_cos_sim(output_list[14][0], output_list[22][0])

print("몸에 있는배 vs 낙시배 유사도:{}".format(simulate1))
print("몸에 있는배 vs 요트배 유사도:{}".format(simulate2))
print("낙시배 vs 요트배 유사도:{}".format(simulate3))

sequence_output size: torch.Size([31, 1, 768])
몸에 있는배 vs 낙시배 유사도:tensor([[0.6264]])
몸에 있는배 vs 요트배 유사도:tensor([[0.5682]])
낙시배 vs 요트배 유사도:tensor([[0.7463]])
